In [104]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import neighbors
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [105]:
a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")

a = a[['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%', 'Yr', 'Blk%', 'Stl%']]
a[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 
    'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 
    'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%', 'Yr', 'Blk%', 'Stl%']].apply(pd.to_numeric, errors="ignore")
a = a.replace(to_replace='NO IDEA', value=0)
a = a.replace(to_replace='---', value=0)
a = a.fillna(0)
a = a.drop(a.loc[((a['games']>39) & (a['year']==2023)) | ((a['games']>40) & (a['year']==2022)) | 
   ((a['games']>33) & (a['year']==2021)) |  
   ((a['games']>35) & (a['year']==2020)) | 
   ((a['games']>40) & (a['year']==2019)) |
   (a['year']==0)]['Player Name'].index)
a.columns

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_92312/1334117891.py:1: DtypeWarning: Columns (10,13,16,19,35) have mixed types. Specify dtype option on import or set low_memory=False.
  a = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_dataset.csv")


Index(['Player Name', 'season', 'Team', 'conf_abbr', 'games', 'games_started',
       'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct',
       ...
       'G', 'S', '%Min', 'ORtg', '%Poss', '%Shots', 'DR%', 'Yr', 'Blk%',
       'Stl%'],
      dtype='object', length=114)

In [106]:
#a[a['Player Name']=='cam-spencer-1']

In [107]:
#beta_df = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/Prediction Names Full.csv")
beta_df = pd.read_csv("/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/Prediction Names Full Full.csv")
beta_df = beta_df[['PLAYER_ID', 'KP Diff', 'Year Diff', 'Calc A2_23_24', 'Calc B_23_24', 'Team_AdjEM_dif', 'Class_n1']]
beta_df = beta_df.rename({'Class_n1': 'Class Temp'}, axis=1)
year_cond = [(beta_df['Class Temp']=='Fr'), (beta_df['Class Temp']=='So'),
            (beta_df['Class Temp']=='Jr'), (beta_df['Class Temp']=='Sr')]
year_val = [1, 2, 3, 4]
beta_df['Class_n1'] = np.select(year_cond, year_val)
#beta_df


In [108]:
beta_df = pd.merge(beta_df, a, left_on='PLAYER_ID', right_on='Player Name', how='inner')
beta_df = beta_df[beta_df['season']=='2022-23']

beta_df = beta_df.rename({'KP Diff': 'KP_Conf_Dif', 'Year Diff': 'Year_dif'}, axis=1)
beta_df['Calc A'] = beta_df['%Min']*beta_df['ws']
beta_df['Calc A2'] = beta_df['%Min']*beta_df['bpm']
beta_df['Calc A3'] = beta_df['%Min']*beta_df['ORtg']
year_cond = [(beta_df['Yr']=='Fr'), (beta_df['Yr']=='So'),
            (beta_df['Yr']=='Jr'), (beta_df['Yr']=='Sr')]
year_val = [1, 2, 3, 4]
beta_df['Class'] = np.select(year_cond, year_val)

# beta_df['KP_Conf_Dif'] = beta_df['KP_Conf_Dif']*-1
# beta_df['Team_AdjEM_dif'] = beta_df['Team_AdjEM_dif']*-1

beta_df['Calc B'] = (beta_df['stl_per_poss']+beta_df['blk_per_poss']+beta_df['DR%'])*beta_df['%Min']
beta_df['Calc C']=(beta_df['off_rtg']-beta_df['def_rtg'])*beta_df['%Min']

beta_df['Calc Overall A'] = beta_df['Calc A'] + beta_df['Calc B']
beta_df['Calc Overall A2'] = beta_df['Calc A2'] + beta_df['Calc B']
beta_df['Calc Overall A3'] = beta_df['Calc A3'] + beta_df['Calc B']

beta_df['player_type'] = 0

beta_df = beta_df.reset_index()
#beta_df

In [109]:
# All In One Go,
kp = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KenPom Team Metrics.csv')
#kp.head(10)
kp_conf_rat = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/KP_Conf_Ratings.csv')
kp_conf_rat = kp_conf_rat[['Year_Conf', 'Conf_Abb', 'Conf_Rk', 'Conf_Full', 'Conf_Rating']]
# Prep initial df,
d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
d = pd.merge(d, kp, left_on=['Team', 'year'], right_on=['TeamName', 'Season'], how='left')
d = pd.merge(d, kp, left_on=['Team_n1', 'year'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(d, kp, left_on=['Team_n1', 'year_n1'], right_on=['TeamName', 'Season'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = pd.merge(df, kp_conf_rat, left_on=['conf_abbr_n1', 'year_n1'], right_on=['Conf_Abb', 'Year_Conf'], how='left')
df = df.replace(to_replace='NO IDEA', value=0)
df = df.replace(to_replace='---', value=0)
df = df.fillna(0)

# Convert to Numeric,
df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 
    'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
    'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 
    'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors='ignore')

# Add Contributions,
conditions = [(df['X.Poss_n1']>=28.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=24.0)& (df['X.Min_n1']>10.0), 
              (df['X.Poss_n1']>=20.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=16) & (df['X.Min_n1']>10.0),
            (df['X.Poss_n1']>=12.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']<12) & (df['X.Min_n1']>10.0), 
            (df['X.Min_n1']<10)]
#(df['X.Min_n1']<10),,
#values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer'],
values = [0, 1, 2, 3, 4, 5, 6]
df['Contribution_n1'] = np.select(conditions, values)

# Add Year in School KP,
year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'),
            (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
year_val = [1, 2, 3, 4]
df['Class'] = np.select(year_cond, year_val)

# Add Ht. Categories,
ht_cond = [(df['Ht']<=74), (df['Ht']>74) & (df['Ht']<=76), (df['Ht']>76) & (df['Ht']<=78),
   (df['Ht']>78) & (df['Ht']<=80), (df['Ht']>80)]

ht_val = [1, 2, 3, 4, 5]
df['Pos_Ht'] = np.select(ht_cond, ht_val)
#df[‘column’] =( df[‘column’] – df[‘column’].mean() ) / df[‘column’].std(),

    
# Additional Features + Model Prep,
df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>=0.1)].index)
df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>=0.1)].index)
#df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>0.1)].index)
#df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>0.1)].index)


df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
   ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
   ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
   ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
   ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
   (df['year']==0)]['Player.Name'].index)
idx = df.groupby(['Team', 'year'])['games'].max()
a = pd.DataFrame(idx)
ab = a.reset_index()
ab.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab, left_on=['Team', 'year'], right_on=['Team_max', 'year_max'], how='left')
idx2 = df.groupby(['Team_n1', 'year_n1'])['games_n1'].max()
a2 = pd.DataFrame(idx2)
ab2 = a2.reset_index()
ab2.columns = ['Team_max', 'year_max', 'games_max']
df = pd.merge(df, ab2, left_on=['Team_n1', 'year_n1'], right_on=['Team_max', 'year_max'], how='left')
df.columns
tt = df[['Player.Name', 'Team', 'year', 'games_max_x', 'Team_n1', 'year_n1', 'games_max_y', 'games_n1']]
tt[tt['Player.Name']=='tevin-mack-1']
#df = df.drop(df[(df['games']<=0.1*df['games_max_x'])].index)
#df = df.drop(df[(df['games_n1']<=0.1*df['games_max_y'])].index)
#df = df.drop(df.loc[(df['games_max_x']<15)].index)
#df = df.drop(df.loc[(df['games_max_y']<15)].index)

#df = df.drop(df.loc[(df['year']==2020) | (df['year_n1']==2020)].index)
df = df.drop(df[(df['X.Min_n1']<=10)].index)
df = df.drop(df[(df['X.Min']<=10)].index)
df['Year_dif'] = df['year_n1']-df['year']
df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
df['Calc A'] = df['X.Min']*(df['ws']-min(df['ws']))
df['Calc A2'] = df['X.Min']*df['bpm']
# df['Calc A2'] = df['bpm']/(1-df['X.Min'])
df['Calc A3'] = df['X.Min']*df['ORtg']
df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1']-min(df['ws_n1']))
df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
df['Calc Overall A'] = df['Calc A'] + df['Calc B']
df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

# Calc_A_n1_cond = [(df['Calc A_n1']<=340), (df['Calc A_n1']>340) & (df['Calc A_n1']<=664), (df['Calc A_n1']>664) & (df['Calc A_n1']<=1010),,
#            (df['Calc A_n1']>1010) & (df['Calc A_n1']<=1415), (df['Calc A_n1']>1415)],

# Calc_A_n1_val = [0, 1, 2, 3, 4],
# df['Calc_A_Contr'] = np.select(Calc_A_n1_cond, Calc_A_n1_val),
column_to_test = 'Calc A2_n1'
# Calc_overall_n1_cond = [(df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()), ,
#                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()) & (df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()), ,
#                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()) & (df[column_to_test]<=8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()),,
#                         (df[column_to_test]>8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min())],

                        # & (df[column_to_test]<=4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min()), ,
                        #(df[column_to_test]>4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min())],
Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.1)),
                        (df[column_to_test]>df[column_to_test].quantile(0.1)) & (df[column_to_test]<=df[column_to_test].quantile(0.4)),
                        (df[column_to_test]>df[column_to_test].quantile(0.4)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
                        (df[column_to_test]>df[column_to_test].quantile(0.88))]
# Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.5)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.5)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.88))]
#Calc_A_n1_val = [2, 1, 0]
Calc_A_n1_val = [3, 2, 1, 0]
df['Calc_Overall_n1_Contr'] = np.select(Calc_overall_n1_cond, Calc_A_n1_val)

/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_92312/4094357533.py:7: DtypeWarning: Columns (13,16,38,144,147,169) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv('/Users/caldavis/Documents/GitHub/Capstone-project/Data folder/final_data_by_player_type.csv')
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_92312/4094357533.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Contribution_n1'] = np.select(conditions, values)
/var/folders/cq/16hkxgp926d466c0g9jzx2900000gq/T/ipykernel_92312/4094357533.py:38: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usi

In [110]:
#df[df['player_type']==0]

In [111]:

# # Convert to Numeric,
# df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 
#     'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 
#     'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 
#     'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']] = df[['games', 'games_started', 'mp_per_g', 'fg_per_g', 'fga_per_g', 'fg_pct', 'fg2_per_g', 'fg2a_per_g', 'fg2_pct', 'fg3_per_g', 'fg3a_per_g', 'fg3_pct', 'ft_per_g', 'fta_per_g', 'ft_pct', 'orb_per_g', 'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g', 'tov_per_g', 'pf_per_g', 'pts_per_g', 'sos', 'mp', 'fg', 'fga', 'fg2', 'fg2a', 'fg2_pct.1', 'fg3', 'fg3a', 'ft', 'fta', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'fg_per_min', 'fga_per_min', 'fg2_per_min', 'fg2a_per_min', 'fg3_per_min', 'fg3a_per_min', 'ft_per_min', 'fta_per_min', 'trb_per_min', 'ast_per_min', 'stl_per_min', 'blk_per_min', 'tov_per_min', 'pf_per_min', 'pts_per_min', 'fg_per_poss', 'fga_per_poss', 'fg2_per_poss', 'fg2a_per_poss', 'fg3_per_poss', 'fg3a_per_poss', 'ft_per_poss', 'fta_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss', 'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss', 'off_rtg', 'def_rtg', 'per', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ows', 'dws', 'ws', 'ws_per_40', 'obpm', 'dbpm', 'bpm', 'year', 'Ht', 'Wt', 'G', 'S', 'X.Min', 'ORtg', 'X.Poss', 'X.Shots', 'games_n1', 'games_started_n1', 'mp_per_g_n1', 'fg_per_g_n1', 'fga_per_g_n1', 'fg_pct_n1', 'fg2_per_g_n1', 'fg2a_per_g_n1', 'fg2_pct_n1', 'fg3_per_g_n1', 'fg3a_per_g_n1', 'fg3_pct_n1', 'ft_per_g_n1', 'fta_per_g_n1', 'ft_pct_n1', 'orb_per_g_n1', 'drb_per_g_n1', 'trb_per_g_n1', 'ast_per_g_n1', 'stl_per_g_n1', 'blk_per_g_n1', 'tov_per_g_n1', 'pf_per_g_n1', 'pts_per_g_n1', 'sos_n1', 'mp_n1', 'fg_n1', 'fga_n1', 'fg2_n1', 'fg2a_n1', 'fg2_pct.1_n1', 'fg3_n1', 'fg3a_n1', 'ft_n1', 'fta_n1', 'orb_n1', 'drb_n1', 'trb_n1', 'ast_n1', 'stl_n1', 'blk_n1', 'tov_n1', 'pf_n1', 'pts_n1', 'fg_per_min_n1', 'fga_per_min_n1', 'fg2_per_min_n1', 'fg2a_per_min_n1', 'fg3_per_min_n1', 'fg3a_per_min_n1', 'ft_per_min_n1', 'fta_per_min_n1', 'trb_per_min_n1', 'ast_per_min_n1', 'stl_per_min_n1', 'blk_per_min_n1', 'tov_per_min_n1', 'pf_per_min_n1', 'pts_per_min_n1', 'fg_per_poss_n1', 'fga_per_poss_n1', 'fg2_per_poss_n1', 'fg2a_per_poss_n1', 'fg3_per_poss_n1', 'fg3a_per_poss_n1', 'ft_per_poss_n1', 'fta_per_poss_n1', 'trb_per_poss_n1', 'ast_per_poss_n1', 'stl_per_poss_n1', 'blk_per_poss_n1', 'tov_per_poss_n1', 'pf_per_poss_n1', 'pts_per_poss_n1', 'off_rtg_n1', 'def_rtg_n1', 'per_n1', 'ts_pct_n1', 'efg_pct_n1', 'fg3a_per_fga_pct_n1', 'fta_per_fga_pct_n1', 'pprod_n1', 'orb_pct_n1', 'drb_pct_n1', 'trb_pct_n1', 'ast_pct_n1', 'stl_pct_n1', 'blk_pct_n1', 'tov_pct_n1', 'usg_pct_n1', 'ows_n1', 'dws_n1', 'ws_n1', 'ws_per_40_n1', 'obpm_n1', 'dbpm_n1', 'bpm_n1', 'year_n1', 'Ht_n1', 'Wt_n1', 'G_n1', 'S_n1', 'X.Min_n1', 'ORtg_n1', 'X.Poss_n1', 'X.Shots_n1']].apply(pd.to_numeric, errors='ignore')

# # Add Contributions,
# conditions = [(df['X.Poss_n1']>=28.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=24.0)& (df['X.Min_n1']>10.0), 
#               (df['X.Poss_n1']>=20.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']>=16) & (df['X.Min_n1']>10.0),
#             (df['X.Poss_n1']>=12.0) & (df['X.Min_n1']>10.0), (df['X.Poss_n1']<12) & (df['X.Min_n1']>10.0), 
#             (df['X.Min_n1']<10)]
# #(df['X.Min_n1']<10),,
# #values = ['Go-To Guy', 'Major Contributor', 'Significant Contributor', 'Role Players', 'Limited Role', 'Nearly Invisible', 'Bench Warmer'],
# values = [0, 1, 2, 3, 4, 5, 6]
# df['Contribution_n1'] = np.select(conditions, values)

# # Add Year in School KP,
# year_cond = [(df['Yr_n1']=='Fr'), (df['Yr_n1']=='So'),
#             (df['Yr_n1']=='Jr'), (df['Yr_n1']=='Sr')]
# year_val = [1, 2, 3, 4]
# df['Class'] = np.select(year_cond, year_val)

# # Add Ht. Categories,
# ht_cond = [(df['Ht']<=74), (df['Ht']>74) & (df['Ht']<=76), (df['Ht']>76) & (df['Ht']<=78),
#    (df['Ht']>78) & (df['Ht']<=80), (df['Ht']>80)]

# ht_val = [1, 2, 3, 4, 5]
# df['Pos_Ht'] = np.select(ht_cond, ht_val)
# #df[‘column’] =( df[‘column’] – df[‘column’].mean() ) / df[‘column’].std(),

    
# # Additional Features + Model Prep,
# df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>=0.1)].index)
# df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>=0.1)].index)
# #df = df.drop(df[(df['games']<=13) & (df['games_started']/df['games']>0.1)].index)
# #df = df.drop(df[(df['games_n1']<=13) & (df['games_started_n1']/df['games_n1']>0.1)].index)


# df = df.drop(df.loc[((df['games']>39) & (df['year']==2023)) | ((df['games_n1']>39) & (df['year_n1']==2023)) | 
#    ((df['games']>40) & (df['year']==2022)) | ((df['games_n1']>40) & (df['year_n1']==2022)) | 
#    ((df['games']>33) & (df['year']==2021)) | ((df['games_n1']>33) & (df['year_n1']==2021)) | 
#    ((df['games']>35) & (df['year']==2020)) | ((df['games_n1']>35) & (df['year_n1']==2020)) | 
#    ((df['games']>40) & (df['year']==2019)) | ((df['games_n1']>40) & (df['year_n1']==2019)) |
#    (df['year']==0)]['Player.Name'].index)
# # idx = df.groupby(['Team', 'year'])['games'].max()
# # a = pd.DataFrame(idx)
# # ab = a.reset_index()
# # ab.columns = ['Team_max', 'year_max', 'games_max']
# # df = pd.merge(df, ab, left_on=['Team', 'year'], right_on=['Team_max', 'year_max'], how='left')
# # idx2 = df.groupby(['Team_n1', 'year_n1'])['games_n1'].max()
# # a2 = pd.DataFrame(idx2)
# # ab2 = a2.reset_index()
# # ab2.columns = ['Team_max', 'year_max', 'games_max']
# # df = pd.merge(df, ab2, left_on=['Team_n1', 'year_n1'], right_on=['Team_max', 'year_max'], how='left')
# # df.columns
# # tt = df[['Player.Name', 'Team', 'year', 'games_max_x', 'Team_n1', 'year_n1', 'games_max_y', 'games_n1']]
# # tt[tt['Player.Name']=='tevin-mack-1']
# #df = df.drop(df[(df['games']<=0.1*df['games_max_x'])].index)
# #df = df.drop(df[(df['games_n1']<=0.1*df['games_max_y'])].index)
# #df = df.drop(df.loc[(df['games_max_x']<15)].index)
# #df = df.drop(df.loc[(df['games_max_y']<15)].index)

# #df = df.drop(df.loc[(df['year']==2020) | (df['year_n1']==2020)].index)
# df = df.drop(df[(df['X.Min_n1']<=10)].index)
# df = df.drop(df[(df['X.Min']<=10)].index)
# df['Year_dif'] = df['year_n1']-df['year']
# df['Team_AdjEM_dif'] = df['AdjEM_y']-df['AdjEM_x']
# df['KP_Rk_Dif'] = df['RankAdjEM_y']-df['RankAdjEM_x']
# df['KP_Conf_Dif'] = df['Conf_Rating_y']-df['Conf_Rating_x']
# df['Calc A'] = df['X.Min']*(df['ws']-min(df['ws']))
# df['Calc A2'] = df['X.Min']*df['bpm']
# # df['Calc A2'] = df['bpm']/(1-df['X.Min'])
# df['Calc A3'] = df['X.Min']*df['ORtg']
# df['Calc A2_n1'] = df['X.Min_n1']*df['bpm_n1']
# # df['Calc A2_n1'] = df['bpm_n1']/(1-df['X.Min_n1'])
# df['Calc A_n1'] =df['X.Min_n1']*(df['ws_n1']-min(df['ws_n1']))
# df['Calc A3_n1'] = df['X.Min_n1']*df['ORtg_n1']
# df['Calc B'] = (df['stl_per_poss']+df['blk_per_poss']+df['DR.'])*df['X.Min']
# df['Calc B_n1'] = (df['stl_per_poss_n1']+df['blk_per_poss_n1']+df['DR._n1'])*df['X.Min_n1']
# df['Calc C']=(df['off_rtg']-df['def_rtg'])*df['X.Min']
# df['Calc C_n1']=(df['off_rtg_n1']-df['def_rtg_n1'])*df['X.Min_n1']
# df['Calc Overall A'] = df['Calc A'] + df['Calc B']
# df['Calc Overall A2'] = df['Calc A2'] + df['Calc B']
# df['Calc Overall A3'] = df['Calc A3'] + df['Calc B']
# df['Calc Overall A_n1'] = df['Calc A_n1'] + df['Calc B_n1']
# df['Calc Overall A2_n1'] = df['Calc A2_n1'] + df['Calc B_n1']
# df['Calc Overall A3_n1'] = df['Calc A3_n1'] + df['Calc B_n1']

# # Calc_A_n1_cond = [(df['Calc A_n1']<=340), (df['Calc A_n1']>340) & (df['Calc A_n1']<=664), (df['Calc A_n1']>664) & (df['Calc A_n1']<=1010),,
# #            (df['Calc A_n1']>1010) & (df['Calc A_n1']<=1415), (df['Calc A_n1']>1415)],

# # Calc_A_n1_val = [0, 1, 2, 3, 4],
# # df['Calc_A_Contr'] = np.select(Calc_A_n1_cond, Calc_A_n1_val),
# column_to_test = 'Calc A2_n1'
# # Calc_overall_n1_cond = [(df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()), ,
# #                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()) & (df[column_to_test]<=(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()), ,
# #                         (df[column_to_test]>(df[column_to_test].max()-df[column_to_test].min())/2+df[column_to_test].min()) & (df[column_to_test]<=8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min()),,
# #                         (df[column_to_test]>8*(df[column_to_test].max()-df[column_to_test].min())/10+df[column_to_test].min())],

#                         # & (df[column_to_test]<=4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min()), ,
#                         #(df[column_to_test]>4*(df[column_to_test].max()-df[column_to_test].min())/6+df[column_to_test].min())],
# Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.1)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.1)) & (df[column_to_test]<=df[column_to_test].quantile(0.4)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.4)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
#                         (df[column_to_test]>df[column_to_test].quantile(0.88))]
# # Calc_overall_n1_cond = [(df[column_to_test]<=df[column_to_test].quantile(0.5)),
# #                         (df[column_to_test]>df[column_to_test].quantile(0.5)) & (df[column_to_test]<=df[column_to_test].quantile(0.88)),
# #                         (df[column_to_test]>df[column_to_test].quantile(0.88))]
# #Calc_A_n1_val = [2, 1, 0]
# Calc_A_n1_val = [3, 2, 1, 0]
# df['Calc_Overall_n1_Contr'] = np.select(Calc_overall_n1_cond, Calc_A_n1_val)

In [112]:
#df['Calc A2_n1'].quantile(0.1)

In [113]:
pd.set_option('display.max_columns', None)
#beta_df

In [114]:
df_test1 = df[['bpm', 'Calc A2', 'ws', 
                 'KP_Conf_Dif', 'X.Poss', 'Year_dif',  'Calc A', 'Calc B', 'Calc A3', 
                 'DR.',  'Team_AdjEM_dif', 'player_type', 'Contribution_n1',
                 'Class', 'Calc C', 'ORtg',
                 'Calc_Overall_n1_Contr',  
                 'per', 'efg_pct', 'fta_per_fga_pct', 'pprod', 'orb_pct', 'usg_pct', 'tov_pct', 'Calc Overall A', 'Calc Overall A2', 'Calc Overall A3']]
seed = 7
test_size = 0.16
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)
#model_test = GaussianNB()
model_test = GradientBoostingClassifier()
#X_temp = df_test1[keep_col]

#3 Bin Testing
#X_temp = df_test1[['player_type', 'Calc A2', 'ORtg', 'KP_Conf_Dif', 'orb_pct', 'Calc A3']]
#4 Bin Testing
X_temp = df_test1[['player_type', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]

#X_temp = df_test1[['player_type', 'Calc A2', 'bpm', 'Calc C', 'DR.', 'KP_Conf_Dif', 'Class']]
#X_temp.head(10)

y_temp = df_test1[['Calc_Overall_n1_Contr']]
X_train_check, X_test_check, y_train_check, y_test_check = train_test_split(X_temp, y_temp, test_size=test_size, random_state=seed)

model_test.fit(X_train_check, y_train_check)
model_test.score(X_test_check, y_test_check)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.6260720411663808

In [115]:
beta_X_prep = beta_df[['player_type', 'Calc A2', 'Calc B', 'bpm', 'tov_pct', 'efg_pct', 'ws', 'Year_dif']]
#beta_X_prep.rename({'DR%': 'DR.'}, axis=1)
predictions = model_test.predict(beta_X_prep)
#predictions

In [116]:
#beta_df

In [117]:
beta_df['fg3_pct'] = beta_df['fg3_pct'].astype(float)
beta_df['ft_pct'] = beta_df['ft_pct'].astype(float)
# for name in ['player_type',  'sos', 'trb_per_poss',
#              'ast_per_poss', 'pts_per_poss', 'Ht', 'Calc A2', 'Calc A3',
#               'Year_dif', 'Team_AdjEM_dif', 'KP_Conf_Dif', 'Class_n1', 'fg3_pct', 'ft_pct']
    #print(name, beta_df[name].dtypes)

#'Class_n1', 'fg3_pct', 'ft_pct',

In [118]:
beta_df['Predicted Offense for 23-24'] = -329+0.008076*beta_df['Class_n1']+27.9*beta_df['fg3_pct']+26.51*beta_df['ft_pct']+29.07*beta_df['player_type']+9.91*beta_df['sos']+6.01*beta_df['trb_per_poss']+9.11*beta_df['ast_per_poss']+1.869*beta_df['pts_per_poss']+2.344*beta_df['Ht']+0.555*beta_df['Calc A2']+0.008076*beta_df['Calc A3']+0.01845*beta_df['Year_dif']+2.033*beta_df['Team_AdjEM_dif']+2.709*beta_df['KP_Conf_Dif']
beta_df['Predicted Defense for 23-24'] = -889.16+50.09115*beta_df['player_type']+0.3256*beta_df['Calc B']+1.68908*beta_df['games']+6.81469*beta_df['mp_per_g']+11.48689*beta_df['per']-11.31937*beta_df['Class_n1']-11.26411*beta_df['Team_AdjEM_dif']-10.41892*beta_df['KP_Conf_Dif']+64.04556*beta_df['dws']+10.83994*beta_df['Ht']+30.3512*beta_df['Stl%']
#0.008076*beta_df['Class_n1']++27.9*beta_df['fg3_pct']+26.51*beta_df['ft_pct']

In [119]:
pred = pd.DataFrame(predictions)
pd.set_option('display.max_columns', None)
#beta_df = y_test_check.reset_index()
pred = pd.merge(pred, beta_df, left_index=True, right_index=True)
pred = pred[['PLAYER_ID', 0, 'Calc A2_23_24', 'Predicted Offense for 23-24', 'Calc B_23_24', 'Predicted Defense for 23-24']]

#pred

In [120]:
pred_col = 'Calc A2_23_24'

# 3-Bin
# pred_overall_n1_cond = [(pred[pred_col]<=df[column_to_test].quantile(0.5)),
#                         (pred[pred_col]>df[column_to_test].quantile(0.5)) & (pred[pred_col]<=df[column_to_test].quantile(0.88)),
#                         (pred[pred_col]>df[column_to_test].quantile(0.88))]

# pred_A_n1_val = ['Role Player', 'Starter', 'Star Player']

# 4-Bin
pred_overall_n1_cond = [(pred[pred_col]<=df[column_to_test].quantile(0.1)),
                        (pred[pred_col]>df[column_to_test].quantile(0.1)) & (pred[pred_col]<=df[column_to_test].quantile(0.4)),
                        (pred[pred_col]>df[column_to_test].quantile(0.4)) & (pred[pred_col]<=df[column_to_test].quantile(0.88)),
                        (pred[pred_col]>df[column_to_test].quantile(0.88))]



pred_A_n1_val = ['Non-Factor', 'Role Player', 'Starter', 'Star Player']

pred['Grouping'] = np.select(pred_overall_n1_cond, pred_A_n1_val)
#pred[pred['Calc A2_23_24']!=0]

In [121]:
g_matrix = pd.read_csv('/Users/caldavis/Desktop/Guard Matrix.csv', index_col='Player')
f_matrix = pd.read_csv('/Users/caldavis/Desktop/Forward Matrix.csv', index_col='Player')
c_matrix = pd.read_csv('/Users/caldavis/Desktop/Center Matrix.csv', index_col='Player')

In [122]:
sim_dict = {'PLAYER_ID_SEASON': [], '1st': [], '2nd': [], '3rd': [], '4th': [], '5th': []}
for name in pred['PLAYER_ID']:
    name = name + "_2023"
    sim_dict['PLAYER_ID_SEASON'].append(name)
    if name in list(g_matrix.index):
        a = list(g_matrix[name].sort_values(ascending=False).index[1:6])
        b = list(g_matrix[name].sort_values(ascending=False)[1:6])
    elif name in list(f_matrix.index):
        a = list(f_matrix[name].sort_values(ascending=False).index[1:6])
        b = list(f_matrix[name].sort_values(ascending=False)[1:6])
    elif name in list(c_matrix.index):
        a = list(c_matrix[name].sort_values(ascending=False).index[1:6])
        b = list(c_matrix[name].sort_values(ascending=False)[1:6])
    else:
        print(name)
        print("bad")
    sim_dict['1st'].append(a[0] + " - " + str(round(100*b[0], 1)) + "%")
    sim_dict['2nd'].append(a[1] + " - " + str(round(100*b[1], 1)) + "%")
    sim_dict['3rd'].append(a[2] + " - " + str(round(100*b[2], 1)) + "%")
    sim_dict['4th'].append(a[3] + " - " + str(round(100*b[3], 1)) + "%")
    sim_dict['5th'].append(a[4] + " - " + str(round(100*b[4], 1)) + "%")


sim_df = pd.DataFrame.from_dict(sim_dict)

In [123]:
#pred = pred.reset_index()
pred = pd.merge(pred, sim_df, left_index=True, right_index=True)
pred = pred.rename({'Calc A2_23_24': 'Actual Offense 23-24', 'Calc B_23_24': 'Actual Defense 23-24', 'Grouping': 'Actual Overall Contribution 23-24'}, axis=1)
year_cond = [(pred[0]==0), (pred[0]==1), 
             (pred[0]==2), (pred[0]==3)]
year_val = ['Star Player', 'Starter', 'Role Player', 'Non-Factor']
pred['Predicted Overall Contribution 23-24'] = np.select(year_cond, year_val)
pred = pred.sort_values(0)
#pred

In [124]:
final_pred_df = pred[['PLAYER_ID', 'Predicted Overall Contribution 23-24', 'Predicted Offense for 23-24', 'Predicted Defense for 23-24', 'PLAYER_ID_SEASON', '1st', '2nd', '3rd', '4th', '5th']]
final_pred_df = final_pred_df[final_pred_df['PLAYER_ID'].isin(['cam-spencer-1', 'josh-cohen-1', 'jordan-king-1', 
                                                               'demilade-adelekun-1', 'noah-farrakhan-1', 'terrell-ardjr-1', 'max-shulga-1', 'steven-ashworth-1'])]
final_pred_df


,PLAYER_ID,Predicted Overall Contribution 23-24,Predicted Offense for 23-24,Predicted Defense for 23-24,PLAYER_ID_SEASON,1st,2nd,3rd,4th,5th
0,cam-spencer-1,Star Player,595.501452,1249.010301,cam-spencer-1_2023,michael-flowers-2_2022 - 91.8%,mark-sears-1_2023 - 91.4%,payton-willis-1_2022 - 89.0%,jahmir-young-1_2023 - 87.9%,jalen-pickett-1_2022 - 85.6%
1,josh-cohen-1,Star Player,307.783086,1320.696315,josh-cohen-1_2023,jordan-brown-4_2023 - 88.2%,grant-basile-1_2022 - 87.8%,matt-rogers-1_2023 - 84.3%,drew-timme-1_2023 - 84.3%,grant-nelson-1_2023 - 84.2%
5,demilade-adelekun-1,Star Player,350.107756,1237.900712,demilade-adelekun-1_2023,josh-oduro-1_2023 - 87.3%,amari-williams-2_2023 - 85.9%,trevion-williams-1_2022 - 85.8%,daniel-deaver-1_2023 - 85.5%,jalen-slawson-1_2022 - 83.7%
7,steven-ashworth-1,Star Player,654.121566,643.905694,steven-ashworth-1_2023,collin-gillespie-1_2022 - 94.7%,alex-barcello-1_2022 - 94.4%,souley-boum-1_2022 - 92.9%,kendric-davis-1_2022 - 92.8%,isaiah-stevens-1_2022 - 91.2%
17,max-shulga-1,Star Player,383.996992,929.689233,max-shulga-1_2023,pelle-larsson-1_2023 - 90.6%,jalen-pickett-1_2023 - 90.3%,colby-jones-1_2023 - 89.0%,souley-boum-1_2022 - 88.4%,jaden-ivey-1_2022 - 87.5%
2,jordan-king-1,Starter,-43.463494,843.822272,jordan-king-1_2023,jalen-rucker-1_2022 - 87.8%,jalen-rucker-1_2023 - 87.4%,max-abmas-1_2022 - 86.8%,john-michael-wright-1_2022 - 86.1%,trey-calvin-1_2022 - 84.7%
24,noah-farrakhan-1,Role Player,-212.445201,820.772763,noah-farrakhan-1_2023,brandon-rush-2_2022 - 74.8%,colin-smalls-1_2022 - 73.5%,markeith-browningii-1_2023 - 72.2%,maxwell-land-1_2022 - 69.0%,tarik-cool-1_2022 - 68.9%
25,terrell-ardjr-1,Role Player,-190.017602,996.656078,terrell-ardjr-1_2023,deejuan-pruitt-1_2023 - 72.9%,thomas-howell-1_2023 - 72.0%,fidelis-okereke-1_2023 - 71.4%,malcolm-chimezie-1_2023 - 69.3%,malachi-rhodes-1_2022 - 69.2%


In [125]:
actual_df = pred[['PLAYER_ID', 'Predicted Overall Contribution 23-24', 'Actual Overall Contribution 23-24', 
                  'Actual Offense 23-24', 'Predicted Offense for 23-24', 
                  'Actual Defense 23-24', 'Predicted Defense for 23-24']]
actual_df = actual_df[actual_df['PLAYER_ID'].isin(['cam-spencer-1', 'josh-cohen-1', 'jordan-king-1', 
                                                               'demilade-adelekun-1', 'noah-farrakhan-1', 'terrell-ardjr-1', 'max-shulga-1', 'steven-ashworth-1'])]
actual_df

,PLAYER_ID,Predicted Overall Contribution 23-24,Actual Overall Contribution 23-24,Actual Offense 23-24,Predicted Offense for 23-24,Actual Defense 23-24,Predicted Defense for 23-24
0,cam-spencer-1,Star Player,Star Player,968.78,595.501452,1288.97,1249.010301
1,josh-cohen-1,Star Player,Star Player,452.97,307.783086,1402.05,1320.696315
5,demilade-adelekun-1,Star Player,Starter,215.60,350.107756,940.80,1237.900712
7,steven-ashworth-1,Star Player,Star Player,370.56,654.121566,818.32,643.905694
17,max-shulga-1,Star Player,Star Player,539.24,383.996992,1340.17,929.689233
2,jordan-king-1,Starter,Star Player,385.88,-43.463494,1122.56,843.822272
24,noah-farrakhan-1,Role Player,Role Player,-74.80,-212.445201,1234.20,820.772763
25,terrell-ardjr-1,Role Player,Role Player,-96.36,-190.017602,525.60,996.656078
